# Check accuracy of kNN imputation

In [11]:
import numpy as np
import pandas as pd

from plotting import *
from pprint import pprint

from sklearn.impute import KNNImputer

## Load the known data

In [12]:
pcd_df = pd.read_csv('data/All ML Systems - full view.csv')
pcd_df.head()

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,...,Organization (from Organization),Base model,Finetune compute (FLOP),Finetune compute notes,Authors by country,Hardware quantity,Hardware utilization,Training cost trends,Training cloud compute vendor,Training data center
0,Cohere Command,Language,NaN,NaN,NaN,NaN,NaN,https://cohere.com/models/command,NaN,"World-class AI, at your command",...,Cohere,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Theseus,Other,Maze solving,Claude Shannon,Historical significance,NaN,NaN,https://www.technologyreview.com/2018/12/19/13...,0.0,Mighty Mouse,...,Bell Laboratories,NaN,NaN,NaN,Theseus,NaN,NaN,NaN,NaN,NaN
2,SNARC,Other,Maze solving,Marvin Minsky,Historical significance,NaN,NaN,https://en.wikipedia.org/wiki/Stochastic_neura...,33.0,A Neural-Analogue Calculator Based upon a Prob...,...,Harvard University,NaN,NaN,NaN,SNARC,NaN,NaN,NaN,NaN,NaN
3,Genetic algorithm,NaN,NaN,NA Barricelli,Historical significance,Possibly first computer simulation of a geneti...,NaN,https://link.springer.com/article/10.1007/BF01...,266.0,Numerical testing of evolution theories,...,Institute for Advanced Study,NaN,NaN,NaN,Genetic algorithm,NaN,NaN,NaN,NaN,NaN
4,Sequence-based pattern recognition,Vision,Character recognition,O. G. Selfridge,Historical significance,NaN,NaN,https://dl.acm.org/doi/10.1145/1455292.1455310,290.0,Pattern recognition and modern computers,...,Massachusetts Institute of Technology (MIT),NaN,NaN,NaN,Sequence-based pattern recognition,NaN,NaN,NaN,NaN,NaN


In [13]:
# Publication date in datetime format
pcd_df.dropna(subset=['Publication date'], inplace=True)
pcd_df['Publication date'] = pd.to_datetime(pcd_df['Publication date'])

In [14]:
# Manually copied from "Training cost trends" Airtable
frontier_systems = [
    "PaLM 2",
    "GPT-4",
    "Minerva (540B)",
    "Megatron-Turing NLG 530B",
    "GPT-3 175B (davinci)",
    "Meena",
    "AlphaStar",
    "AlphaGo Zero",
    "AlphaGo Master",
    "GNMT",
    "Claude 2",
    "PaLM (540B)",
    "ERNIE 3.0 Titan",
    "Gopher (280B)",
    "OpenAI Five",
    "T5-11B",
    "Megatron-BERT",
    "ResNeXt-101 32x48d",
    "AlphaZero",
    "Falcon 180B",
    "GPT-3.5 (text-davinci-003)",
    "Chinchilla",
    "Yuan 1.0",
    "Turing-NLG",
    "BigGAN-deep 512x512",
    "NASv3 (CIFAR-10)",
    "AlphaGo Lee",
    "AlphaGo Fan",
    "OPT-175B",
    "AlphaCode",
    "GLaM",
    "OpenAI Five Rerun",
    "T5-3B",
    "Megatron-LM (8.3B)",
    "FTW",
    "AmoebaNet-A (F=448)",
    "OpenAI TI7 DOTA 1v1",
    "JFT",
    "Llama 2-70B",
    "LLaMA-65B",
    "LaMDA",
    "ALIGN",
    "GShard (dense)",
    "RoBERTa Large",
    "IMPALA",
]

In [15]:
frontier_pcd_df = pcd_df[pcd_df['System'].isin(frontier_systems)]
frontier_pcd_df.head()

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,...,Organization (from Organization),Base model,Finetune compute (FLOP),Finetune compute notes,Authors by country,Hardware quantity,Hardware utilization,Training cost trends,Training cloud compute vendor,Training data center
265,AlphaGo Fan,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",SOTA improvement,NaN,NaN,https://www.nature.com/articles/nature24270.ep...,14389.0,Mastering the game of Go with deep neural netw...,...,Google DeepMind,NaN,NaN,NaN,AlphaGo Fan,NaN,NaN,AlphaGo Fan,NaN,NaN
275,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14389.0,Mastering the game of Go with deep neural netw...,...,DeepMind,NaN,NaN,NaN,AlphaGo Lee,NaN,NaN,AlphaGo Lee,NaN,NaN
306,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1609.08144,5948.0,Google's Neural Machine Translation System: Br...,...,Google,NaN,NaN,NaN,GNMT,96.0,NaN,GNMT,NaN,NaN
317,NASv3 (CIFAR-10),Vision,NaN,"Barret Zoph, Quoc V. Le",Highly cited,NaN,NaN,https://arxiv.org/abs/1611.01578,4569.0,Neural Architecture Search with Reinforcement ...,...,Google Brain,NaN,NaN,NaN,NASv3 (CIFAR-10),800.0,NaN,NASv3 (CIFAR-10),NaN,NaN
337,AlphaGo Master,Games,Go,"D Silver, J Schrittwieser, K Simonyan, I Anton...",Highly cited,NaN,NaN,https://www.researchgate.net/publication/32047...,7831.0,Mastering the game of Go without human knowledge,...,DeepMind,NaN,NaN,NaN,AlphaGo Master,NaN,NaN,AlphaGo Master,NaN,NaN


In [16]:
pcd_hardware_model_colname = 'Name of the hardware (from Training hardware)'

In [17]:
chip_hours = []
for i, row in frontier_pcd_df.iterrows():
    if pd.isna(row['Hardware quantity']) or pd.isna(row['Training time (hours)']):
        # TODO impute missing values
        chip_hours.append(np.nan)
    else:
        chip_hours.append(row['Hardware quantity'] * row['Training time (hours)'])

frontier_pcd_df['Training time (chip hours)'] = chip_hours

C:\Users\robir\AppData\Local\Temp\ipykernel_22536\3951382658.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frontier_pcd_df['Training time (chip hours)'] = chip_hours


## Prepare for imputation

In [18]:
# drop unneeded columns from frontier_pcd_df
irrelevant_columns = ['Notability criteria', 'Notability criteria notes', 'Link', 'Citations', 'Parameters notes',
                      'Training compute notes', 'Training dataset notes', 'Dataset size notes',
                      'Inference compute notes', 'Approach', 'Confidence', 'Last modified', 'Created By', 'Benchmark data',
                      'Exclude', 'Authors by country', 'Training cost trends', 'Abstract', 'Compute cost notes',
                      'Training time notes', 'Authors', 'Name of the hardware (from Training hardware)',
                      'Training time (chip hours)', 'Training compute cost (2020 USD)', 'Organization categorization',
                      'Training dataset', 'Inference compute (FLOP)', 'Compute sponsor categorization',
                      'Finetune compute (FLOP)', 'Finetune compute notes', ]
frontier_pcd_df = frontier_pcd_df.drop(columns=irrelevant_columns)

# fill column 'Training cloud compute vendor' using org_to_cloud_vendor dictionary
org_to_cloud_vendor = {
    'Google': 'Google Cloud',
    'DeepMind': 'Google Cloud',
    'Google DeepMind': 'Google Cloud',
    'Google Brain': 'Google Cloud',
    'Microsoft': 'Microsoft Azure',
    'OpenAI': 'Microsoft Azure',
}
frontier_pcd_df['Training cloud compute vendor'] = frontier_pcd_df['Organization (from Organization)'].map(org_to_cloud_vendor)
frontier_pcd_df['Training cloud compute vendor'] = frontier_pcd_df['Training cloud compute vendor'].fillna('Amazon Web Services')

In [19]:
# convert large number columns to logarithmic
frontier_pcd_df['log_params'] = np.log10(frontier_pcd_df['Parameters'])
frontier_pcd_df['log_compute'] = np.log10(frontier_pcd_df['Training compute (FLOP)'])
frontier_pcd_df['log_dataset'] = np.log10(frontier_pcd_df['Training dataset size (datapoints)'])
# drop raw columns
raw_columns = ['Parameters', 'Training compute (FLOP)', 'Training dataset size (datapoints)']
frontier_pcd_df.drop(columns=raw_columns, inplace=True)

In [20]:
frontier_pcd_df.columns

Index(['System', 'Domain', 'Task', 'Open-source', 'Reference',
       'Publication date', 'Organization', 'Epochs', 'Training time (hours)',
       'Training hardware', 'Country (from Organization)',
       'Organization (from Organization)', 'Base model', 'Hardware quantity',
       'Hardware utilization', 'Training cloud compute vendor',
       'Training data center', 'log_params', 'log_compute', 'log_dataset'],
      dtype='object')

In [25]:
print(
    "Known hardware models:", frontier_pcd_df['Training hardware'].notna().sum(), "\n",
    "Known hardware utilization:", frontier_pcd_df['Hardware utilization'].notna().sum(), "\n",
    "Known hardware quantity:", frontier_pcd_df['Hardware quantity'].notna().sum(), "\n",
    "Known training time:", frontier_pcd_df['Training time (hours)'].notna().sum()
)

Known hardware models: 35 
 Known hardware utilization: 14 
 Known hardware quantity: 33 
 Known training time: 27


Next, for different values of k, perform imputation and calculate the 

## Run imputation many times and evaluate